# Using Selenium and Requests: _Raven+Lily_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Raven+Lily.
First, we will use selenium to get everylink for the bags, and then use Beautiful Soup to get the correct picture,price, product link, and description.

Some great sources used to help in this: 
- [Selenium Website for Locating Elements](https://selenium-python.readthedocs.io/locating-elements.html)
- [Stackover flow to Save Images](https://stackoverflow.com/questions/18497840/beautifulsoup-how-to-open-images-and-download-them)
- [Get all image src's with specific class](https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract)


Our steps are as follows: 
1. [first]()
2. [second]()
3. [third]()
4. [fourth]() 

#### Import your needed Libraries:

In [1]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
import urllib
from PIL import Image
import requests
from io import BytesIO

#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on Raven+Lily.

In [70]:
RL_urls = ['https://ravenandlily.com/collections/backpacks','https://ravenandlily.com/collections/crossbody-bags',
        'https://ravenandlily.com/collections/hobos','https://ravenandlily.com/collections/totes',
        'https://ravenandlily.com/collections/weekenders']

#### Create our Chrome Driver element and get Links:

Here I am using the Chrome Driver Manager because I was facing some issues using the driver, but this resolved all of them.

In [89]:
def BS_scraper(urls):
    img_links_tot = []
    desc_tot = []
    price_tot = []

    for url in urls:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'lxml')

        match = soup.findAll('div', {'class':'ci'})# find all tags with images
        img_links_tot.extend(['https:'+ x.img.findNext('img')['srcset'] for x in match])
        match2 = soup.findAll('div', {'class':'product-details'})
        desc_tot.extend([x.h3.text for x in match2])
        price_tot.extend([x.span.text for x in match2])
        
    bag_dict = {'link':img_links_tot,'description':desc_tot,'price':price_tot}
    return bag_dict
        
    

In [ ]:
RL_dict = BS_scraper(RL_urls)

In [76]:
RL_df = pd.DataFrame(RL_dict)

##### 1. Clean the product description text:

We want to lower case all of the names and get rid of the special characters.

In [79]:
RL_df.description = RL_df.description.apply(lambda x: x.replace('// ','').lower())

##### 2. Change the prices to floats:

In [81]:
RL_df.price = RL_df.price.apply(lambda x: float(x.replace('$','')))

In [83]:
RL_df.to_csv('raven_lily.csv')

__Drop Duplicate Rows based on Column names:__

In [ ]:
RL_df.drop_duplicates(['description'],inplace=True)
RL_df.reset_index(drop=True,inplace=True)

### Download the images

In [138]:
c = 0
def save_src_image_apply(url):
    start_time = time.time()
    time.sleep(1)
    try:
        global c
        c += 1
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        image_save_loc = 'raven_lily/'+'rl_'+str(c)+'.jpg'
        img.save(image_save_loc, "JPEG")
        print("Saved "+"rl_{}".format(c)+".png")
        print(c)
        return "rl_"+ str(c)+".png"
    except Exception as e:
        print(str(e))
        return e

In [139]:
RL_df['img'] = RL_df.link.apply(lambda x: save_src_image_apply(x))

Saved rl_1.png
1
Saved rl_2.png
2
Saved rl_3.png
3
Saved rl_4.png
4
Saved rl_5.png
5
Saved rl_6.png
6
Saved rl_7.png
7
Saved rl_8.png
8
Saved rl_9.png
9
Saved rl_10.png
10
Saved rl_11.png
11
Saved rl_12.png
12
Saved rl_13.png
13
Saved rl_14.png
14
Saved rl_15.png
15
Saved rl_16.png
16
Saved rl_17.png
17
Saved rl_18.png
18
Saved rl_19.png
19
Saved rl_20.png
20
Saved rl_21.png
21
Saved rl_22.png
22
Saved rl_23.png
23
Saved rl_24.png
24
Saved rl_25.png
25
Saved rl_26.png
26
Saved rl_27.png
27
Saved rl_28.png
28
Saved rl_29.png
29
Saved rl_30.png
30
Saved rl_31.png
31
Saved rl_32.png
32
Saved rl_33.png
33
Saved rl_34.png
34
Saved rl_35.png
35
Saved rl_36.png
36
Saved rl_37.png
37
Saved rl_38.png
38
Saved rl_39.png
39
Saved rl_40.png
40
Saved rl_41.png
41
Saved rl_42.png
42
Saved rl_43.png
43


__After further review of the images it became clear that the correct images did not exist. So, we decided to drop those two rows/products.__

In [144]:
RL_df.drop([16,24],inplace=True)

In [145]:
RL_df.to_csv('data/raven_lily.csv')